In [1]:
wet_file_path =('/content/drive/MyDrive/wet.paths')
# reading the file
with open(wet_file_path,'r') as f:
  wet_urls = f.read().splitlines()
  # show first five urls
  print(wet_urls[:5])

['crawl-data/CC-MAIN-2025-30/segments/1751905933612.63/wet/CC-MAIN-20250707183638-20250707213638-00000.warc.wet.gz', 'crawl-data/CC-MAIN-2025-30/segments/1751905933612.63/wet/CC-MAIN-20250707183638-20250707213638-00001.warc.wet.gz', 'crawl-data/CC-MAIN-2025-30/segments/1751905933612.63/wet/CC-MAIN-20250707183638-20250707213638-00002.warc.wet.gz', 'crawl-data/CC-MAIN-2025-30/segments/1751905933612.63/wet/CC-MAIN-20250707183638-20250707213638-00003.warc.wet.gz', 'crawl-data/CC-MAIN-2025-30/segments/1751905933612.63/wet/CC-MAIN-20250707183638-20250707213638-00004.warc.wet.gz']


## download first two files

In [5]:
import requests
base_url = 'https://data.commoncrawl.org/'
# dowload the first wet file to avoid overload
file_url = base_url + wet_urls[0]
# taking the local filename
local_filename = wet_urls[0].split('/')[-1]
r = requests.get(file_url,stream = True)
# opening the file
with open(local_filename,'wb') as f:
  for chunk in r.iter_content(chunk_size = 1024):
    if chunk:
      f.write(chunk)
      print(f"{local_filename} downloaded.")

Streaming output truncated to the last 5000 lines.
CC-MAIN-20250707183638-20250707213638-00000.warc.wet.gz downloaded.
CC-MAIN-20250707183638-20250707213638-00000.warc.wet.gz downloaded.
CC-MAIN-20250707183638-20250707213638-00000.warc.wet.gz downloaded.
CC-MAIN-20250707183638-20250707213638-00000.warc.wet.gz downloaded.
CC-MAIN-20250707183638-20250707213638-00000.warc.wet.gz downloaded.
CC-MAIN-20250707183638-20250707213638-00000.warc.wet.gz downloaded.
CC-MAIN-20250707183638-20250707213638-00000.warc.wet.gz downloaded.
CC-MAIN-20250707183638-20250707213638-00000.warc.wet.gz downloaded.
CC-MAIN-20250707183638-20250707213638-00000.warc.wet.gz downloaded.
CC-MAIN-20250707183638-20250707213638-00000.warc.wet.gz downloaded.
CC-MAIN-20250707183638-20250707213638-00000.warc.wet.gz downloaded.
CC-MAIN-20250707183638-20250707213638-00000.warc.wet.gz downloaded.
CC-MAIN-20250707183638-20250707213638-00000.warc.wet.gz downloaded.
CC-MAIN-20250707183638-20250707213638-00000.warc.wet.gz downloade

In [ ]:
import gzip
import csv

output_csv = "parsed_data.csv"

# Open the downloaded gzipped file for reading
with gzip.open(local_filename, 'rt', encoding='utf-8', errors='ignore') as f:
    with open(output_csv, 'w', newline='', encoding='utf-8') as out_csv:
        writer = csv.writer(out_csv)
        writer.writerow(['url', 'Content'])

        current_url = ""
        current_content = []

        for line in f:
            line = line.strip()

            # New URL found
            if line.startswith('WARC-Target-URI:'):
                # Save the previous URL and its content
                if current_url and current_content:
                    writer.writerow([current_url, " ".join(current_content)])
                # Start new entry
                current_url = line.replace('WARC-Target-URI: ', '')
                current_content = []
            elif line.startswith('WARC/') or line.startswith('Content-Length:') or line.startswith('Content-Type:') or line.startswith('WARC-Type:'):
                continue  # Skip metadata
            else:
                current_content.append(line)

        # Save last URL content
        if current_url and current_content:
            writer.writerow([current_url, " ".join(current_content)])

print(f"Extracted content saved to {output_csv}")


In [7]:
import pandas as pd
cv_file = pd.read_csv('/content/parsed_data.csv')
print(cv_file.head(20))


                                                  url  \
0        http://0481.jp/g/bainanza/performance/20967/   
1   http://0509.kiss206.com/index.phtml?PUT=a_show...   
2       http://083656015.com.tw/front/bin/login.phtml   
3   http://0svbncq.qjlzw.cn/?5906637.html?jincheng...   
4   http://100church.org/home/board.php?board=cast...   
5   http://12zzzzz.com/htm/2025/6/20/renqinvyou/66...   
6   http://14243.live173.com/?PUT=a_show&AID=27784...   
7   http://14243.live173.com/?PUT=a_show&AID=27905...   
8    http://170144.syk003.com/?PUT=up_edit&FID=170144   
9   http://1784634.gt68m.com/index.phtml?PUT=a_sho...   
10  http://192186.007best.com/index.phtml?PUT=A_SO...   
11                       http://1ynx.ru/chan/text/141   
12                       http://200az.com/vodtype/36/   
13  http://201472.appavut.com/?PUT=a_show&AID=2764...   
14  http://2130159.puy047.com/?PUT=a_show&AID=2302...   
15                                http://2237210.com/   
16                          htt

## Keyword se extract

In [10]:
adult_keywords =['porn', 'sex', 'xxx', 'nude', 'boobs', 'camgirl', 'milf', 'hentai', 'blowjob', 'fetish']


def classify(row):
  # lowercase the text
  text = (row['url']+" "+row['Content']).lower()
  for word in adult_keywords:
    if word in text:
      return 'adult'
    return 'safe'
cv_file['label'] = cv_file.apply(classify,axis=1)
cv_file.to_csv("labbeled_data_csv",index=False)

In [12]:
df = pd.read_csv('/content/labbeled_data_csv')
print(df.head())

                                                 url  \
0       http://0481.jp/g/bainanza/performance/20967/   
1  http://0509.kiss206.com/index.phtml?PUT=a_show...   
2      http://083656015.com.tw/front/bin/login.phtml   
3  http://0svbncq.qjlzw.cn/?5906637.html?jincheng...   
4  http://100church.org/home/board.php?board=cast...   

                                             Content label  
0  WARC-Date: 2025-07-07T19:03:53Z WARC-Record-ID...  safe  
1  WARC-Date: 2025-07-07T19:54:36Z WARC-Record-ID...  safe  
2  WARC-Date: 2025-07-07T20:16:05Z WARC-Record-ID...  safe  
3  WARC-Date: 2025-07-07T18:55:49Z WARC-Record-ID...  safe  
4  WARC-Date: 2025-07-07T20:37:22Z WARC-Record-ID...  safe  
